In [ ]:
pip install tqdm

In [ ]:
pip install nltk matplotlib

In [ ]:
import nltk
nltk.download('gutenberg')

In [2]:
import sys
sys.path.append(r'C:\Users\ASUS\TensorFlow Keywords\CBOW_master')

In [ ]:
import os
import nltk
from cbow_model import CBOW
from tqdm import tqdm

# 读取文本数据
input_folder = 'C:/Users/ASUS/TensorFlow Keywords/cleaned_data/'

# 创建 word_vector 文件夹以保存模型文件
output_model_folder = 'C:/Users/ASUS/TensorFlow Keywords/word_vector'
os.makedirs(output_model_folder, exist_ok=True)

# 遍历每个子文件夹中的所有文件
for category in os.listdir(input_folder):
    category_path = os.path.join(input_folder, category)
    if os.path.isdir(category_path):
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            if os.path.isfile(file_path):
                # 读取文件内容
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                # 创建 CBOW 模型实例
                model = CBOW(text=text)

                # 设置训练参数
                window = 2
                n = 100
                learning_rate = 0.0001

                # 分割文本
                split_text = model.text.lower().split()

                # 训练模型
                for center_word in tqdm(split_text, desc='Training', unit='word'):
                    index = split_text.index(center_word)
                    left_context = split_text[max(0, index - window):index]
                    right_context = split_text[index + 1:min(len(split_text), index + window + 1)]
                    avg_vector = model.get_average_vector_context(left_context, right_context)
                    x_hat = model.get_score(avg_vector)
                    y_hat = model.softmax(x_hat)
                    center_word_index = model.word_index[center_word]
                    center_word_one_hot_encoding = model.get_one_hot(center_word_index)
                    error = model.compute_cross_entropy_error(center_word_one_hot_encoding, y_hat)
                    model.update_U(error, avg_vector)
                    model.update_V(error, left_context, right_context)

                # 保存模型文件
                output_model_file = os.path.join(output_model_folder, f"{file_name}.bin")
                model.save_model(output_model_file)


Training: 100%|████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 3509.77word/s]


In [3]:
output_model_file = 'C:/Users/ASUS/TensorFlow Keywords/word_vector/word_vectors_model.bin'
model.save_model(output_model_file)